In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
num_samples = 44100

In [3]:
input1 = linspace(-1, 1, num_samples)
input2 = linspace(1, -1, num_samples)

In [8]:
!cat 01_simple.stride

use DesktopAudio version 1.0

signal Accum {}
switch ResetOuter {}
signal OuterCounter { reset: ResetOuter}

loop Loop {
	ports: [		
	]
	blocks: [
		signal Counter { reset: CounterReset }
		switch CounterReset {}
	]
	streams: [
		Accum + 2 >> Accum;
		Counter + 1 >> Counter;
		[Counter, 10] >> Equal() >> CounterReset;
	]
	terminateWhen: CounterReset
}


[OuterCounter, 20] >> Equal() >> ResetOuter >> Loop(); 
OuterCounter + 1 >> OuterCounter;
Accum >> AudioOut[1];



In [11]:
out_text = ''

outercounter = 0
counter = 0
accum = 0

for val1, val2 in zip(input1, input2):
    if outercounter == 20:
        for i in range(10):
            accum += 2
        outercounter = 0
    outercounter += 1
    out_text += str(accum) + "\n0.0\n"

out_file = open("01_simple.expected", "w")
out_file.write(out_text[:-1])
pass